<a href="https://colab.research.google.com/github/anarkia7115/q_learning/blob/master/Taxi_deep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# !pip install gym

In [0]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe
import gym

In [0]:
def get_model(input_size, nA):

  model = tf.keras.Sequential([
      tf.keras.layers.Dense(10, activation=tf.nn.relu, 
                            input_shape=(input_size,)), 
      tf.keras.layers.Dense(10, activation=tf.nn.relu), 
      tf.keras.layers.Dense(nA)
  ])
  
  return model


In [0]:
def predict_reward(
        env,
        observation,
        a
):
    [row, col, passidx, destidx] = \
        env.decode(observation)
    nR = 5
    nC = 5
    maxR = nR-1
    maxC = nC-1
    taxiloc = (row, col)
    newrow, newcol, newpassidx = row, col, passidx
    reward = -1
    if a==0:
        newrow = min(row+1, maxR)
    elif a==1:
        newrow = max(row-1, 0)
    if a==2 and env.desc[1+row,2*col+2]==b":":
        newcol = min(col+1, maxC)
    elif a==3 and env.desc[1+row,2*col]==b":":
        newcol = max(col-1, 0)
    elif a==4: # pickup
        if (passidx < 4 and taxiloc == env.locs[passidx]):
            newpassidx = 4
        else:
            reward = -10
    elif a==5: # dropoff
        if (taxiloc == env.locs[destidx]) and passidx==4:
            newpassidx = destidx
            done = True
            reward = 20
        elif (taxiloc in env.locs) and passidx==4:
            newpassidx = env.locs.index(taxiloc)
        else:
            reward = -10
    newstate = env.encode(newrow, newcol, newpassidx, destidx)
    return reward, newstate

In [0]:
def max_q(env, nA, state_t):
  max_qp1 = -100
  for a in range(nA):
    _, state_tp1 = predict_reward(
        env.env, state_t, a
    )
    action = tf.argmax(model(tf.convert_to_tensor([state_tp1, a])), axis=1)
    q = tf.max(model(tf.convert_to_tensor([state_tp1, a])))
    if tf.greater(q, max_qp1):
      max_qp1 = q
      selected_a = a
          
  return max_qp1


In [0]:
def loss(model, env, nA, s, a, r, gamma):
  q_star = tf.add(
      r, 
      tf.multiply(
          gamma, 
          max_q(env, nA, s)))
  
  q = tf.max(model(tf.convert_to_tensor([s, a])))
  return tf.square(tf.sub(q_star, q))


In [0]:
def grad(model, env, nA, s, a, r, gamma):
  with tf.GradientTape() as tape:
    loss_value = loss(model, env, nA, s, a, r, gamma)
  return loss_value, tape.gradient(loss_value, model.trainable_variables)

In [0]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
global_step = tf.train.get_or_create_global_step()

In [0]:
env_name = 'Taxi-v2'
env = gym.make(env_name)
status_dim = env.observation_space.n
action_dim = env.action_space.n

model = get_model(1, action_dim)
lr = 0.8
gamma = 0.95
epoch_size = 10
episode_size = 200
nA = 6

for i_epoch in range(epoch_size):
  observation = env.reset()  
  epoch_loss_avg = tfe.metrics.Mean()
  epoch_accuracy = tfe.metrics.Accuracy()
  for t in range(episode_size):
    env.render()
    action = tf.argmax(model(observation), axis=1)

    observation, reward, done, info = \
              env.step(action)

    # update Q
    # grad(model, env, nA, s, a, r, gamma)
    loss_value, grads = grad(model, env, nA, observation, action, reward, gamma)
    optimizer.apply_gradients(zip(grads, model.variables), 
                              global_step)
    # Track progress
    epoch_loss_avg(loss_value)  # add current batch loss
    # compare predicted label to actural label
#     epoch_accuracy(tf.argmax(model(tf.convert_to_tensor([s, a])), 
#                              axis=1, output_type=tf.int32), y)

    if done:
      print("Episode[{}] finished after {} timestamps".format(i_episode, t+1))
      t_list.append(t)
      # t_list.append(t)
      break
  

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



/usr/local/lib/python3.6/dist-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


ValueError: ignored

In [0]:
def grad(model, inputs, targets):
  with tf.GradientTape() as tape:
    loss_value = loss(model, inputs, targets)
  return loss_value, tape.gradient(loss_value, model.trainable_variables)

In [0]:
def loss():
  pass

In [0]:
def train(m):
  # init 